In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import sys
import os

In [2]:
#load data
df = pd.read_csv("/home/julian/Project/code/quora_duplicate_questions.tsv",delimiter='\t')

names=df.columns.values
print(names)
print(df.shape)

y=df['is_duplicate']
print(y.shape)



['id' 'qid1' 'qid2' 'question1' 'question2' 'is_duplicate']
(404290, 6)
(404290,)


In [53]:
y_train=y[0:50000]
y_test=y[50000:100000]

subset=df[0:100000]
#df=subset

#print(df)

In [4]:
#training for embedding
import gensim


In [5]:

df['question1'] = df['question1'].apply(lambda x: unicode(str(x),"utf-8"))
df['question2'] = df['question2'].apply(lambda x: unicode(str(x),"utf-8"))


In [6]:
questions=list(df['question1'])+list(df['question2'])
c=0
for i in tqdm(questions):
    questions[c]=list(gensim.utils.tokenize(i,deacc=True,lower=True))
    c+=1

100%|██████████| 808580/808580 [01:06<00:00, 12123.81it/s]


In [7]:
from gensim.models import Word2Vec
embed_model=Word2Vec(questions,size=300,min_count=10)
print(embed_model['what'])

[  1.47635266e-01   2.02503845e-01  -3.00855905e-01   7.04323828e-01
   5.16311228e-01   2.34611854e-01  -1.06530595e+00   1.79300085e-01
   1.47915792e+00   6.21605814e-01  -5.87303638e-01   3.15053314e-02
  -9.59633589e-02  -2.97713894e-02   4.35157001e-01   4.26067531e-01
   8.58545125e-01   7.41145194e-01   1.21569800e+00  -4.43062276e-01
  -1.55046439e+00   8.58229578e-01   5.79359055e-01   2.00924826e+00
  -1.41297615e+00   1.34151936e+00  -4.69542086e-01   5.12131810e-01
  -2.29192704e-01   2.45426923e-01  -8.73400986e-01   4.35823709e-01
   1.33816898e+00  -8.19411337e-01  -1.21927273e+00   8.05692375e-01
  -1.04685938e+00   9.61022913e-01   3.88170153e-01  -6.20268643e-01
  -1.81212962e+00  -5.67588925e-01  -1.65157542e-01  -7.19464242e-01
  -1.12206340e-01  -1.00563693e+00   1.40891686e-01  -1.09762836e+00
   9.72508788e-01  -5.02483964e-01   5.18619239e-01  -8.78886878e-02
  -2.75892854e-01  -1.11928016e-01   3.51531915e-02  -1.59858990e+00
   9.63451982e-01   1.01817083e+00

In [9]:
import tensorflow as tf

In [10]:
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence,text
from keras.layers import Dense, Input, LSTM, Embedding, Dropout,Merge

Using TensorFlow backend.


In [34]:

tk=text.Tokenizer(filters="")
max_len=40
tk.fit_on_texts(list(subset.question1.values)+list(subset.question2.values))
x1 = tk.texts_to_sequences(subset.question1.values)
x1 = sequence.pad_sequences(x1,maxlen = max_len)

x2 = tk.texts_to_sequences(subset.question2.values)
x2 = sequence.pad_sequences(x1,maxlen = max_len)

In [35]:
q=subset['question1']
print(q[0])
print(x1[0])

What is the step by step guide to invest in share market in india?
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    2    3    1 1464
   53 1464 3206    7  514    8  689  602    8   57]


In [36]:
nb_words=len(tk.word_index)+1
print(nb_words)
#print(subset.question1.values)

92844


In [50]:
x1_train=x1[0:50000]
x1_test=x1[50000:100000]
x2_train=x2[0:50000]
x2_test=x2[50000:100000]


TypeError: 'NoneType' object has no attribute '__getitem__'

In [39]:

print(x1_train.shape)
print(x1_test.shape)
print(y_train.shape)
print(y_test.shape)
#y_train=y_train.reshape((1,10))
#y_test=y_test.reshape((1,10))

(50000, 40)
(50000, 40)
(50000,)
(50000,)


In [40]:


embedding_matrix = np.zeros((nb_words, 300))

for word, i in tk.word_index.items():
    embedding_vector = None 
    if word in embed_model.wv.vocab:
        embedding_vector = embed_model[word]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    
#print(embedding_matrix)

In [41]:
embedding_vecor_length = 300
model1 = Sequential()
model1.add(Embedding(nb_words, embedding_vecor_length,weights=[embedding_matrix], input_length=max_len))
#x1=model1.add(LSTM(100,return_sequences=True))
x1=model1.add(LSTM(100))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model1.summary())
#model1.fit(x1_train, y_train, validation_data=(x1_test, y_test),epochs=2,batch_size=64)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 40, 300)           27853200  
_________________________________________________________________
lstm_10 (LSTM)               (None, 100)               160400    
Total params: 28,013,600
Trainable params: 28,013,600
Non-trainable params: 0
_________________________________________________________________
None


In [42]:
embedding_vecor_length = 300
model2 = Sequential()
model2.add(Embedding(nb_words, embedding_vecor_length,weights=[embedding_matrix], input_length=max_len))
#x2=model2.add(LSTM(100,return_sequences=True))
x2=model2.add(LSTM(100))
#model2.add(Dense(1, activation='sigmoid'))
#model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())
#model2.fit(x2_train, y_train, validation_data=(x2_test, y_test),epochs=2,batch_size=64)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 40, 300)           27853200  
_________________________________________________________________
lstm_11 (LSTM)               (None, 100)               160400    
Total params: 28,013,600
Trainable params: 28,013,600
Non-trainable params: 0
_________________________________________________________________
None


In [43]:
model = Sequential()
model.add(Merge([model1, model2], mode='concat'))
#model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit([x1_train,x2_train],y_train,validation_data=([x1_test,x2_test],y_test),epochs=1,batch_size=100)

/home/julian/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  from ipykernel import kernelapp as app


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_6 (Merge)              (None, 200)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 100)               20100     
_________________________________________________________________
dense_11 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 101       
Total params: 56,057,501
Trainable params: 56,057,501
Non-trainable params: 0
_________________________________________________________________
None
Train on 50000 samples, validate on 50000 samples
Epoch 1/1
50000/50000 [==============================] - 3311s - loss: 0.5660 - acc: 0.7079 - val_loss: 0.5440 - val_acc: 0.7282


In [44]:
scores = model.evaluate([x1_test,x2_test],y_test,verbose=0)
print(scores)

[0.54395649314880368, 0.72824]


In [49]:
x1_test=x1[0:100]
#x2_test=x2[60000:60100]
#y_test=y[60000:60100]


TypeError: 'NoneType' object has no attribute '__getitem__'

In [52]:
y_pred=model.predict([x1[100000:100100],x2[100000:100100]],batch_size=2,verbose=0)
count=0
for i in range(100):
    if(y_pred[i]>=0.5):
        res=1
    else:
        res=0
    print(res)
    print(y_test[50000+i])
    if(res==y_test[50000+i]):
        count=count+1
    print(' ')
print(count)

TypeError: 'NoneType' object has no attribute '__getitem__'

In [33]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(x1,x2)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [52]:
embedding_layer = Embedding(nb_words,
                                10,
                                weights=[embedding_matrix],
                                input_length=30,
                                trainable=False)
print(embedding_layer)
lstm_layer = LSTM(75, recurrent_dropout=0.2)

NameError: name 'embedding_matrix' is not defined

In [27]:
MAX_SEQUENCE_LENGTH=30
sequence_1_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1 = lstm_layer(embedded_sequences_1)

sequence_2_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype="int32")
embedded_sequences_2 = embedding_layer(sequence_2_input)
y1 = lstm_layer(embedded_sequences_2)

In [33]:
HIDDEN_DIM=30
EMBED_DIM=10
seq_maxlen=30
ques1_enc = Sequential()
ques1_enc.add(Embedding(output_dim=EMBED_DIM, input_dim=nb_words,
                       weights=[embedding_matrix], mask_zero=True))
ques1_enc.add(LSTM(HIDDEN_DIM, input_shape=(EMBED_DIM, seq_maxlen), 
                       return_sequences=False))
ques1_enc.add(Dropout(0.3))
    
ques2_enc = Sequential()
ques2_enc.add(Embedding(output_dim=EMBED_DIM, input_dim=nb_words,
                       weights=[embedding_matrix], mask_zero=True))
ques2_enc.add(LSTM(HIDDEN_DIM, input_shape=(EMBED_DIM, seq_maxlen), 
                       return_sequences=False))
ques2_enc.add(Dropout(0.3))

In [34]:
model = Sequential()
model.add(Merge([ques1_enc, ques2_enc], mode="sum"))
model.add(Dense(2, activation="softmax"))
    
model.compile(optimizer="adam", loss="categorical_crossentropy",
                  metrics=["accuracy"])
    

/home/julian/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  from ipykernel import kernelapp as app


In [ ]:
checkpoint = ModelCheckpoint(
        filepath=os.path.join(MODEL_DIR, "quora_dul_best_lstm.hdf5"),
        verbose=1, save_best_only=True)
    model.fit([x_ques1train, x_ques2train], ytrain, batch_size=BATCH_SIZE,
              epochs=NBR_EPOCHS, 
              validation_split=0.1,
              verbose=2,
              callbacks=[checkpoint])
    

In [58]:
#model.fit([data_1, data_2], y_train, batch_size=2,epochs=1)
print(y_train)
model.fit(embedding_matrix,y_train,batch_size=2,epochs=1)  

[0 0 0 0 0 1 0 1 0 0]


ValueError: The model expects 2 input arrays, but only received one array. Found: array with shape (122, 10)